# Customer Segmentation with RFM

The dataset named Online Retail II includes online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes souvenirs and it is known that most of its customers are wholesalers.


>1. **InvoiceNo**: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
2. **StockCode**: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
3. **Description**: Product (item) name. Nominal.
4. **Quantity**: The quantities of each product (item) per transaction. Numeric.
5. **InvoiceDate**: Invice date and time. Numeric. The day and time when a transaction was generated.
6. **UnitPrice**: Unit price. Numeric. Product price per unit in sterling (Â£).
7. **CustomerID**: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
8. **Country**: Country name. Nominal. The name of the country where a customer resides.



Dataset link
https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
    


import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df=pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2010-2011")

### Data Understanding

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [5]:
def check_df(dataframe, head=5):
    print("--- Shape ---")
    print(dataframe.shape)
    print("--- Types ---")
    print(dataframe.dtypes)
    print("--- Head ---")
    print(dataframe.head(head))
    print("--- Tail ---")
    print(dataframe.tail(head))
    print("--- NA ---")
    print(dataframe.isnull().sum())
    print("--- Quantiles ---")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [6]:
check_df(df)

--- Shape ---
(541910, 8)
--- Types ---
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
--- Head ---
  Invoice StockCode                          Description  Quantity  \
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                  WHITE METAL LANTERN         6   
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00  2.550    17850.000  United Kingdom  
1 2010-12-01 08:26:00  3.390    17850.000  United Kingdom  
2 2010-12-01 08:26:00  2.750    17850.000  United Kingdom  
3 2010-12-01 08:26:00  3.390    

In [7]:
df["Description"].nunique()

4223

There are 4223 unique products in the dataset

In [8]:
df["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2369
REGENCY CAKESTAND 3 TIER               2200
JUMBO BAG RED RETROSPOT                2159
PARTY BUNTING                          1727
LUNCH BAG RED RETROSPOT                1638
                                       ... 
Missing                                   1
historic computer difference?....se       1
DUSTY PINK CHRISTMAS TREE 30CM            1
WRAP BLUE RUSSIAN FOLKART                 1
PINK BERTIE MOBILE PHONE CHARM            1
Name: Description, Length: 4223, dtype: int64

In [10]:
df.groupby("Description").agg({"Quantity": "sum"})

,Quantity
Description,
20713,-400
4 PURPLE FLOCK DINNER CANDLES,144
50'S CHRISTMAS GIFT BAG LARGE,1913
DOLLY GIRL BEAKER,2448
I LOVE LONDON MINI BACKPACK,389
...,...
wrongly marked carton 22804,-256
wrongly marked. 23343 in box,-3100
wrongly sold (22719) barcode,170


Having negative values of Quantity is a problem we need to solve. There are returns and these returns are expressed in the invoice variable starting with C.

In [11]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [12]:
df["Invoice"].nunique()

25900

In [13]:
df[df["Invoice"].str.contains("C", na=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.500,14527.000,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.650,15311.000,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.650,17548.000,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.830,14397.000,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.690,15498.000,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.950,15311.000,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.250,17315.000,United Kingdom


Returns are excluded from the dataset

In [14]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [15]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    134697
Country             0
dtype: int64

Those who do not have a customer ID should be removed from the data set because we are performing customer segmentation, a customer ID is required. Since there are not too many missing in the description, all the missing in the data set is deleted.

In [16]:
df.dropna(inplace=True)

In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.000,13.022,180.420,1.000,2.000,6.000,12.000,80995.000
Price,397925.000,3.116,22.097,0.000,1.250,1.950,3.750,8142.750
Customer ID,397925.000,15294.309,1713.173,12346.000,13969.000,15159.000,16795.000,18287.000


I prefer not to delete the outliers because when rfm scores are done the outliers will be at a score of 5.

In [21]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

Since the price is the unit price in the invoice, the total price is found by multiplying the quantity with the price.

## Calculating RFM Metrics

In [18]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

Since the newest data in the data set belongs to 2011, I assign today 2 days after the latest date when calculating the recency.

In [19]:
today_date = dt.datetime(2011, 12, 11)

In [22]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [23]:
rfm.columns = ['recency', 'frequency', 'monetary']

In [24]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,326,1,77183.600
12347.000,3,7,4310.000
12348.000,76,4,1797.240
12349.000,19,1,1757.550
12350.000,311,1,334.400


In [25]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4339.000,93.041,100.008,1.000,18.000,51.000,142.500,374.000
frequency,4339.000,4.272,7.705,1.000,1.000,2.000,5.000,210.000
monetary,4339.000,2053.797,8988.248,0.000,307.245,674.450,1661.640,280206.020


Monetary refers to the money that the customer left us.

So 0 is not the value we want to see.

In [26]:
rfm = rfm[rfm["monetary"] > 0]

## Calculating RFM Scores

In [27]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

In [28]:
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [29]:
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

In [30]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.000,326,1,77183.600,1,1,5
12347.000,3,7,4310.000,5,5,5
12348.000,76,4,1797.240,2,4,4
12349.000,19,1,1757.550,4,1,4
12350.000,311,1,334.400,1,1,2


In [31]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [32]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4338.000,93.059,100.012,1.000,18.000,51.000,142.750,374.000
frequency,4338.000,4.273,7.706,1.000,1.000,2.000,5.000,210.000
monetary,4338.000,2054.271,8989.230,3.750,307.415,674.485,1661.740,280206.020


In [33]:
rfm[rfm["RF_SCORE"] == "55"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12347.000,3,7,4310.000,5,5,5,55
12362.000,4,10,5226.230,5,5,5,55
12417.000,4,9,3649.100,5,5,5,55
12423.000,1,8,1859.310,5,5,4,55
12433.000,1,7,13375.870,5,5,5,55
...,...,...,...,...,...,...,...
18230.000,10,7,2810.200,5,5,5,55
18241.000,10,17,2073.090,5,5,5,55
18245.000,8,7,2567.060,5,5,5,55


In [35]:
rfm[rfm["RF_SCORE"] == "31"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12357.000,34,1,6207.670,3,1,5,31
12398.000,46,1,1635.660,3,1,4,31
12403.000,50,1,427.700,3,1,2,31
12420.000,64,1,600.390,3,1,3,31
12430.000,44,1,256.400,3,1,2,31
...,...,...,...,...,...,...,...
15677.000,50,1,657.230,3,1,3,31
15678.000,53,1,352.700,3,1,2,31
15739.000,67,1,449.120,3,1,2,31


### RFM Segments

In [36]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [38]:
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [40]:
rfm.head(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.000,326,1,77183.600,1,1,5,11,hibernating
12347.000,3,7,4310.000,5,5,5,55,champions
12348.000,76,4,1797.240,2,4,4,24,at_Risk
12349.000,19,1,1757.550,4,1,4,41,promising
12350.000,311,1,334.400,1,1,2,11,hibernating
12352.000,37,8,2506.040,3,5,5,35,loyal_customers
12353.000,205,1,89.000,1,1,1,11,hibernating
12354.000,233,1,1079.400,1,1,4,11,hibernating
12355.000,215,1,459.400,1,1,2,11,hibernating


In [41]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.312   352     1.162   352  471.994   352
at_Risk             153.786   593     2.879   593 1084.535   593
cant_loose          132.968    63     8.381    63 2796.156    63
champions             6.362   633    12.417   633 6857.964   633
hibernating         217.605  1071     1.102  1071  488.643  1071
loyal_customers      33.608   819     6.480   819 2864.248   819
need_attention       52.428   187     2.326   187  897.628   187
new_customers         7.429    42     1.000    42  388.213    42
potential_loyalists  17.399   484     2.010   484 1041.222   484
promising            23.511    94     1.000    94  294.008    94

In [42]:
rfm[rfm["segment"] == "about_to_sleep"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12357.000,34,1,6207.670,3,1,5,31,about_to_sleep
12371.000,45,2,1887.960,3,2,4,32,about_to_sleep
12394.000,64,2,1272.480,3,2,4,32,about_to_sleep
12397.000,36,2,2409.900,3,2,5,32,about_to_sleep
12398.000,46,1,1635.660,3,1,4,31,about_to_sleep


In [43]:
rfm[rfm["segment"] == "champions"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12347.000,3,7,4310.000,5,5,5,55,champions
12362.000,4,10,5226.230,5,5,5,55,champions
12364.000,8,4,1313.100,5,4,4,54,champions
12381.000,5,5,1845.310,5,4,4,54,champions
12417.000,4,9,3649.100,5,5,5,55,champions
